In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
compiled_data = pd.read_csv('https://raw.githubusercontent.com/tshunyu/covid-wbe/main/compiled_data.csv')
compiled_data['ww_date'] = pd.to_datetime(compiled_data['ww_date'],format='%Y-%m-%d')

res_df = pd.read_csv('https://raw.githubusercontent.com/tshunyu/covid-wbe/main/results.csv')
res_df['start_date'] = pd.to_datetime(res_df['start_date'],format='%Y-%m-%d')
res_df['end_date'] = pd.to_datetime(res_df['end_date'],format='%Y-%m-%d')
res_df.set_index('wwtp_id',inplace=True)

for ind,_ in res_df.iterrows():
    sample_df = compiled_data[compiled_data['wwtp_id']==ind].sort_values(by=['ww_date'])
    res_df.loc[ind,'total_cases'] = sample_df['cases'].sum()
    res_df.loc[ind,'total_gc'] = (sample_df['population_served']*sample_df['pcr_conc_gc/cap/d']).sum()
    sm,rd = res_df.loc[ind,['sm','rd']]
    sm = 0.46e12 # update this with the reference actual value
    S = sm
    for i in range(1,28):
        S += sm*(1-rd)**i
    res_df.loc[ind,'S'] = S
    res_df.loc[ind,'estimated_cases'] = res_df.loc[ind,'total_gc']/S
    res_df.loc[ind,'silent_cases'] = res_df.loc[ind,'estimated_cases'] - res_df.loc[ind,'total_cases']
    if (res_df.loc[ind,'silent_cases']<0) or (res_df.loc[ind,'total_cases']==0):
        res_df.loc[ind,'ratio'] = 0
    else:
        res_df.loc[ind,'ratio'] = res_df.loc[ind,'silent_cases']/res_df.loc[ind,'total_cases']

In [5]:
res_df[res_df.state=='Pennsylvania']

,county_fips,county,state,population_served,sample_location,normalization_method,per_capita_flow_rate,start_date,end_date,days,...,lt,rd,k,half_life,total_cases,total_gc,S,estimated_cases,silent_cases,ratio
wwtp_id,,,,,,,,,,,,,,,,,,,,,
1667,42043,Dauphin,Pennsylvania,125000,Treatment plant,microbial,6.185157e-09,2022-08-02,2022-12-13,133,...,14,0.430,0.562119,1.232835,1269.197651,2.327500e+04,1.069767e+12,2.175707e-08,-1269.197651,0.000000
1366,42043,Dauphin,Pennsylvania,124000,Treatment plant,flow-population,7.948435e+03,2022-07-04,2022-12-07,156,...,14,0.404,0.517515,1.339093,1555.524824,1.726197e+16,1.138613e+12,1.516052e+04,13604.991583,8.746239
1359,42049,Erie,Pennsylvania,200000,Treatment plant,flow-population,1.085075e+04,2022-07-05,2022-12-06,154,...,0,0.500,0.693147,0.999788,5651.407556,1.772308e+16,9.200000e+11,1.926422e+04,13612.807900,2.408746
1367,42055,Franklin,Pennsylvania,67000,Treatment plant,flow-population,6.430980e+02,2022-07-05,2022-12-06,154,...,0,0.500,0.693147,0.999788,2705.958133,4.847237e+15,9.200000e+11,5.268736e+03,2562.778171,0.947087
1529,42063,Indiana,Pennsylvania,28809,Treatment plant,flow-population,2.913936e+03,2022-07-04,2022-12-07,156,...,8,0.376,0.471605,1.469450,875.512888,3.445704e+15,1.223402e+12,2.816493e+03,1940.980489,2.216964
1617,42069,Lackawanna,Pennsylvania,88000,Treatment plant,flow-population,5.300840e+02,2022-09-28,2022-12-14,77,...,14,0.500,0.693147,0.999788,1044.965896,3.651928e+15,9.200000e+11,3.969487e+03,2924.521045,2.798676
1357,42091,Montgomery,Pennsylvania,30000,Treatment plant,flow-population,3.118730e+03,2022-07-10,2022-12-06,149,...,9,0.383,0.482886,1.435121,394.118178,2.951353e+15,1.201043e+12,2.457325e+03,2063.206890,5.234995
1081,42091,Montgomery,Pennsylvania,13000,Treatment plant,flow-population,3.657052e+03,2022-05-02,2022-12-14,226,...,14,0.482,0.657780,1.053544,136.665631,2.294252e+15,9.543568e+11,2.403977e+03,2267.311854,16.590212
1083,42091,Montgomery,Pennsylvania,10000,Before treatment plant,flow-population,4.699996e+03,2022-05-02,2022-12-14,226,...,2,0.455,0.606969,1.141738,80.867237,2.085375e+15,1.010989e+12,2.062708e+03,1981.840871,24.507340


In [7]:
res_df[(res_df.state=='Pennsylvania')&(res_df['loss']<10)&(res_df['normalization_method']=='flow-population')&(res_df['total_cases']!=0)]\
    [['county','population_served','sample_location','days','sm','lt','rd','total_cases','silent_cases','ratio']]

,county,population_served,sample_location,days,sm,lt,rd,total_cases,silent_cases,ratio
wwtp_id,,,,,,,,,,
1366,Dauphin,124000,Treatment plant,156,4.025019e+12,14,0.404,1555.524824,13604.991583,8.746239
1529,Indiana,28809,Treatment plant,156,1.525588e+12,8,0.376,875.512888,1940.980489,2.216964
1617,Lackawanna,88000,Treatment plant,77,1.558004e+12,14,0.500,1044.965896,2924.521045,2.798676
1357,Montgomery,30000,Treatment plant,149,2.798817e+12,9,0.383,394.118178,2063.206890,5.234995
1081,Montgomery,13000,Treatment plant,226,7.366050e+12,14,0.482,136.665631,2267.311854,16.590212
1083,Montgomery,10000,Before treatment plant,226,1.065301e+13,2,0.455,80.867237,1981.840871,24.507340
1454,Bucks,55000,Treatment plant,121,1.954684e+12,14,0.414,886.762743,3230.600105,3.643139
1369,Westmoreland,25000,Treatment plant,141,3.059261e+12,0,0.383,284.763064,1716.072566,6.026317
1376,Westmoreland,4600,Treatment plant,149,6.066102e+12,10,0.300,10.188626,232.580735,22.827487


In [8]:
res_df[(res_df['loss']<10)&(res_df['normalization_method']=='flow-population')&(res_df['total_cases']!=0)].sm.describe()

count    1.950000e+02
mean     8.459468e+12
std      2.694218e+13
min      9.250961e+09
25%      1.147671e+12
50%      2.637421e+12
75%      6.651443e+12
max      3.519948e+14
Name: sm, dtype: float64